In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.metrics as metrics
import seaborn as sns
from sklearn.model_selection import train_test_split

2024-02-05 04:48:14.940511: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 04:48:14.963034: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 04:48:14.963057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 04:48:14.963073: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 04:48:14.967544: I tensorflow/core/platform/cpu_feature_g

In [2]:
def train_val_test_split(X, y, train_size, val_size, test_size):
    X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, train_size=train_size, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=test_size/(test_size+val_size), stratify=y_val_test)
    return X_train, y_train, X_val, y_val, X_test, y_test

def process_data_for_conv2D(X):
    X_conv2D = []
    for sample in X:
        sample = np.reshape(sample, newshape=(sample.shape[0], sample.shape[1], 1))
        X_conv2D.append(sample)
    return np.array(X_conv2D, dtype=np.float32)

def data_iter(X, y, batch_size):
    n_samples = X.shape[0]
    idx = list(range(n_samples))
    while True:
        for i in range(0, n_samples, batch_size):
            j = idx[i: min(i+batch_size, n_samples)]
            yield X[j, :], y[j, : ]


In [23]:
# define some blocks for NN

def base_conv_block(n_conv_filters, kernel_size):
    def f(input_):
        x = tf.keras.layers.Conv2D(n_conv_filters, kernel_size, padding='same')(input_)


        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        return x
    return f

def base_model_cnn(input_shape, num_conv_filters = [32, 64, 128], kernel_size = (2, 2), max_pool_kernel = (1, 3), num_dense_units=128, num_classes=2):
    input_ = tf.keras.layers.Input(shape=input_shape)
    x = input_
    for n_conv_filters in num_conv_filters:
        x = base_conv_block(n_conv_filters, kernel_size)(x)
        x = tf.keras.layers.MaxPooling2D(max_pool_kernel)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(num_dense_units, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_, outputs=output)
    return model
    

In [27]:
# load data

# X_stft = np.load("stft/stft_features.npy")
# y = np.load("stft/labels.npy")

# # process data for conv2d
# X_stft = process_data_for_conv2D(X_stft)
# print(X_stft.shape)
# print(y.shape)

# load normalized data
X_stft_normalized = np.load("stft_normalized/stft_features.npy")
y = np.load("stft_normalized/labels.npy")

# process data for conv2d
X_stft_normalized = process_data_for_conv2D(X_stft_normalized)
print(X_stft_normalized.shape)
print(y.shape)

(3855, 188, 257, 1)
(3855, 2)


In [28]:
kernel_size = (3, 3)
max_pool_kernel = (1, 4)
conv_filters = [32, 64, 128, 256]
num_dense_units = 512
num_classes = 2
batch_size = 32

train_size = 0.7
val_size = 0.15
test_size = 0.15

epochs = 100
learning_rate = 0.01

# keep track of best model and log
base_file_name = "cnn_model.hdf5"
path = "weights_normalized/"
base_csv_name = "cnn_logs.csv"

# create directory if not exist
if not os.path.exists(path):
    print(f"Create directory {path}")
    os.makedirs(path)

train_loss_record = []
train_acc_record = []
val_loss_record = []
val_acc_record = []
test_loss_record = []
test_acc_record = []

# prepare for k-fold cross validation
k_fold = 5


def kfold_training(X, y, k_fold, train_size, val_size, test_size, base_model_cnn, base_file_name, base_csv_name, path, learning_rate, epochs, batch_size, num_dense_units, num_classes, kernel_size, max_pool_kernel, conv_filters):
    train_loss_record = []
    train_acc_record = []
    val_loss_record = []
    val_acc_record = []
    test_loss_record = []
    test_acc_record = []
    train_auc_record = []
    val_auc_record = []
    test_auc_record = []
    train_precision_record = []
    val_precision_record = []
    test_precision_record = []
    train_recall_record = []
    val_recall_record = []
    test_recall_record = []
    # train_f1_score_record = []
    # val_f1_score_record = []
    # test_f1_score_record = []
    # train_specificity_record = []
    # val_specificity_record = []
    # test_specificity_record = []
    
    for i in range(k_fold):
        print(f"Start {i+1}th-fold in {k_fold} cross validation")

        X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y, train_size, val_size, test_size)

        file_name = os.path.join(path, f"{i}_fold_{base_file_name}")
        csv_path = os.path.join(path, f"{i}_fold_{base_csv_name}")


        lr_change = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, min_lr=0.0001)

        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(file_name, monitor="val_accuracy", save_best_only=True, mode="max", metric="val_acc")

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.01, patience=10, mode="min")

        # csv logger
        csv_logger = tf.keras.callbacks.CSVLogger(csv_path)

        callbacks = [lr_change, model_checkpoint, early_stopping, csv_logger]
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

        model = base_model_cnn(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
                                num_dense_units=num_dense_units, num_classes=num_classes,
                                kernel_size=kernel_size, max_pool_kernel=max_pool_kernel, 
                                num_conv_filters=conv_filters)
        
        # monitor specificity, sensitivity, f1 score
        model.compile(loss = "categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", "AUC", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                validation_data=(X_val, y_val), callbacks=callbacks,
                verbose=1)
        
        # train_loss, train_acc, train_auc, train_precision, train_recall, train_f1_score, train_specificity = model.evaluate(X_train, y_train, verbose=0)
        # val_loss, val_acc, val_auc, val_precision, val_recall, val_f1_score, val_specificity = model.evaluate(X_val, y_val, verbose=0)
        # test_loss, test_acc, test_auc, test_precision, test_recall, test_f1_score, test_specificity = model.evaluate(X_test, y_test, verbose=0)
        
        train_loss, train_acc, train_auc, train_precision, train_recall = model.evaluate(X_train, y_train, verbose=0)
        val_loss, val_acc, val_auc, val_precision, val_recall = model.evaluate(X_val, y_val, verbose=0)
        test_loss, test_acc, test_auc, test_precision, test_recall= model.evaluate(X_test, y_test, verbose=0)
        
        
        train_loss_record.append(train_loss)
        train_acc_record.append(train_acc)
        val_loss_record.append(val_loss)
        val_acc_record.append(val_acc)
        test_loss_record.append(test_loss)
        test_acc_record.append(test_acc)
        train_auc_record.append(train_auc)
        val_auc_record.append(val_auc)
        test_auc_record.append(test_auc)
        train_precision_record.append(train_precision)
        val_precision_record.append(val_precision)
        test_precision_record.append(test_precision)
        train_recall_record.append(train_recall)
        val_recall_record.append(val_recall)
        test_recall_record.append(test_recall)
        # train_f1_score_record.append(train_f1_score)
        # val_f1_score_record.append(val_f1_score)
        # test_f1_score_record.append(test_f1_score)
        # train_specificity_record.append(train_specificity)
        # val_specificity_record.append(val_specificity)
        # test_specificity_record.append(test_specificity)
        

    train_loss_avg = np.mean(train_loss_record)
    train_acc_avg = np.mean(train_acc_record)
    val_loss_avg = np.mean(val_loss_record)
    val_acc_avg = np.mean(val_acc_record)
    test_loss_avg = np.mean(test_loss_record)
    test_acc_avg = np.mean(test_acc_record)
    train_auc_avg = np.mean(train_auc_record)
    val_auc_avg = np.mean(val_auc_record)
    test_auc_avg = np.mean(test_auc_record)
    train_precision_avg = np.mean(train_precision_record)
    val_precision_avg = np.mean(val_precision_record)
    test_precision_avg = np.mean(test_precision_record)
    train_recall_avg = np.mean(train_recall_record)
    val_recall_avg = np.mean(val_recall_record)
    test_recall_avg = np.mean(test_recall_record)
    # train_f1_score_avg = np.mean(train_f1_score_record)
    # val_f1_score_avg = np.mean(val_f1_score_record)
    # test_f1_score_avg = np.mean(test_f1_score_record)
    # train_specificity_avg = np.mean(train_specificity_record)
    # val_specificity_avg = np.mean(val_specificity_record)
    # test_specificity_avg = np.mean(test_specificity_record)
    

    print(f"{k_fold}-fold cv train loss avg: {train_loss_avg:.4f}, train acc avg: {train_acc_avg:.4f}, val loss avg: {val_loss_avg:.4f}, val acc avg: {val_acc_avg:.4f}, test loss avg: {test_loss_avg:.4f}, test acc avg: {test_acc_avg:.4f} \n \
            train auc avg: {train_auc_avg:.4f}, val auc avg: {val_auc_avg:.4f}, test auc avg: {test_auc_avg:.4f} \n \
                train precision avg: {train_precision_avg:.4f}, val precision avg: {val_precision_avg:.4f}, test precision avg: {test_precision_avg:.4f} \n \
                    train recall avg: {train_recall_avg:.4f}, val recall avg: {val_recall_avg:.4f}, test recall avg: {test_recall_avg:.4f}")
                        # train f1 score avg: {train_f1_score_avg:.4f}, val f1 score avg: {val_f1_score_avg:.4f}, test f1 score avg: {test_f1_score_avg:.4f} \n \
                        #     train specificity avg: {train_specificity_avg:.4f}, val specificity avg: {val_specificity_avg:.4f}, test specificity avg: {test_specificity_avg:.4f}")
    



Create directory weights_normalized/


In [29]:
# perform k-fold cross validation
kfold_training(X_stft_normalized, y, k_fold, train_size, val_size, test_size, base_model_cnn, base_file_name, base_csv_name, path, learning_rate, epochs, batch_size, num_dense_units, num_classes, kernel_size, max_pool_kernel, conv_filters)

Start 1th-fold in 5 cross validation
Epoch 1/100
85/85 [==============================] - 4s 29ms/step - loss: 76.4929 - accuracy: 0.6501 - auc: 0.6619 - precision_11: 0.6501 - recall_11: 0.6501 - val_loss: 82.7118 - val_accuracy: 0.5969 - val_auc: 0.5969 - val_precision_11: 0.5969 - val_recall_11: 0.5969 - lr: 0.0100


/home/g6/anaconda3/envs/mech/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
85/85 [==============================] - 2s 28ms/step - loss: 1.2754 - accuracy: 0.7357 - auc: 0.7636 - precision_11: 0.7357 - recall_11: 0.7357 - val_loss: 1.7336 - val_accuracy: 0.6678 - val_auc: 0.7241 - val_precision_11: 0.6678 - val_recall_11: 0.6678 - lr: 0.0100
Epoch 3/100
85/85 [==============================] - 2s 29ms/step - loss: 0.5957 - accuracy: 0.7557 - auc: 0.8295 - precision_11: 0.7557 - recall_11: 0.7557 - val_loss: 0.7909 - val_accuracy: 0.6972 - val_auc: 0.7979 - val_precision_11: 0.6972 - val_recall_11: 0.6972 - lr: 0.0100
Epoch 4/100
85/85 [==============================] - 2s 28ms/step - loss: 0.4834 - accuracy: 0.8336 - auc: 0.8977 - precision_11: 0.8336 - recall_11: 0.8336 - val_loss: 0.3537 - val_accuracy: 0.8581 - val_auc: 0.9292 - val_precision_11: 0.8581 - val_recall_11: 0.8581 - lr: 0.0100
Epoch 5/100
85/85 [==============================] - 2s 28ms/step - loss: 0.3008 - accuracy: 0.8795 - auc: 0.9453 - precision_11: 0.8795 - recall_11: 0.8795 

/home/g6/anaconda3/envs/mech/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 4s 30ms/step - loss: 102.8160 - accuracy: 0.6075 - auc: 0.6177 - precision_12: 0.6075 - recall_12: 0.6075 - val_loss: 1.1918 - val_accuracy: 0.6194 - val_auc: 0.6950 - val_precision_12: 0.6194 - val_recall_12: 0.6194 - lr: 0.0100
Epoch 2/100
85/85 [==============================] - 2s 25ms/step - loss: 0.8401 - accuracy: 0.6475 - auc: 0.7026 - precision_12: 0.6475 - recall_12: 0.6475 - val_loss: 0.8530 - val_accuracy: 0.4066 - val_auc: 0.4636 - val_precision_12: 0.4066 - val_recall_12: 0.4066 - lr: 0.0100
Epoch 3/100
85/85 [==============================] - 2s 29ms/step - loss: 0.6542 - accuracy: 0.6931 - auc: 0.7329 - precision_12: 0.6931 - recall_12: 0.6931 - val_loss: 0.7990 - val_accuracy: 0.6661 - val_auc: 0.7227 - val_precision_12: 0.6661 - val_recall_12: 0.6661 - lr: 0.0100
Epoch 4/100
85/85 [==============================] - 2s 29ms/step - loss: 0.6589 - accuracy: 0.7228 - auc: 0.7616 - precision_12: 0.7228 - recall_12: 0.7228 - val_loss

...

85/85 [==============================] - 2s 25ms/step - loss: 0.7859 - accuracy: 0.7554 - auc: 0.8139 - precision_13: 0.7554 - recall_13: 0.7554 - val_loss: 3.1659 - val_accuracy: 0.3339 - val_auc: 0.4028 - val_precision_13: 0.3339 - val_recall_13: 0.3339 - lr: 0.0100
Epoch 3/100
85/85 [==============================] - 2s 29ms/step - loss: 0.3420 - accuracy: 0.8658 - auc: 0.9300 - precision_13: 0.8658 - recall_13: 0.8658 - val_loss: 0.9842 - val_accuracy: 0.7595 - val_auc: 0.7903 - val_precision_13: 0.7595 - val_recall_13: 0.7595 - lr: 0.0100
Epoch 4/100
85/85 [==============================] - 2s 29ms/step - loss: 0.3108 - accuracy: 0.8851 - auc: 0.9446 - precision_13: 0.8851 - recall_13: 0.8851 - val_loss: 0.4340 - val_accuracy: 0.7907 - val_auc: 0.8873 - val_precision_13: 0.7907 - val_recall_13: 0.7907 - lr: 0.0100
Epoch 5/100
85/85 [==============================] - 2s 25ms/step - loss: 0.2461 - accuracy: 0.9073 - auc: 0.9642 - precision_13: 0.9073 - recall_13: 0.9073 - val_loss: 

/home/g6/anaconda3/envs/mech/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 4s 30ms/step - loss: 122.8837 - accuracy: 0.6490 - auc: 0.6666 - precision_14: 0.6490 - recall_14: 0.6490 - val_loss: 81.2394 - val_accuracy: 0.5969 - val_auc: 0.5969 - val_precision_14: 0.5969 - val_recall_14: 0.5969 - lr: 0.0100
Epoch 2/100
85/85 [==============================] - 2s 29ms/step - loss: 0.9885 - accuracy: 0.7298 - auc: 0.8016 - precision_14: 0.7298 - recall_14: 0.7298 - val_loss: 3.6986 - val_accuracy: 0.6505 - val_auc: 0.6390 - val_precision_14: 0.6505 - val_recall_14: 0.6505 - lr: 0.0100
Epoch 3/100
85/85 [==============================] - 2s 29ms/step - loss: 0.5219 - accuracy: 0.7950 - auc: 0.8718 - precision_14: 0.7950 - recall_14: 0.7950 - val_loss: 1.3581 - val_accuracy: 0.7249 - val_auc: 0.6476 - val_precision_14: 0.7249 - val_recall_14: 0.7249 - lr: 0.0100
Epoch 4/100
85/85 [==============================] - 2s 28ms/step - loss: 0.3958 - accuracy: 0.8225 - auc: 0.9059 - precision_14: 0.8225 - recall_14: 0.8225 - val_los

/home/g6/anaconda3/envs/mech/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 2s 25ms/step - loss: 0.7170 - accuracy: 0.7557 - auc: 0.7824 - precision_15: 0.7557 - recall_15: 0.7557 - val_loss: 0.8484 - val_accuracy: 0.4844 - val_auc: 0.4726 - val_precision_15: 0.4844 - val_recall_15: 0.4844 - lr: 0.0100
Epoch 3/100
85/85 [==============================] - 2s 28ms/step - loss: 0.5217 - accuracy: 0.7721 - auc: 0.8258 - precision_15: 0.7721 - recall_15: 0.7721 - val_loss: 0.5585 - val_accuracy: 0.7301 - val_auc: 0.8010 - val_precision_15: 0.7301 - val_recall_15: 0.7301 - lr: 0.0100
Epoch 4/100
85/85 [==============================] - 2s 28ms/step - loss: 0.5513 - accuracy: 0.7717 - auc: 0.8265 - precision_15: 0.7717 - recall_15: 0.7717 - val_loss: 0.5415 - val_accuracy: 0.7785 - val_auc: 0.8361 - val_precision_15: 0.7785 - val_recall_15: 0.7785 - lr: 0.0100
Epoch 5/100
85/85 [==============================] - 2s 28ms/step - loss: 0.8240 - accuracy: 0.7880 - auc: 0.8385 - precision_15: 0.7880 - recall_15: 0.7880 - val_loss: 

In [ ]:
# # for each patient in patients, do k-fold cross validation
# patients = ["a01r", "a02r", "a03r", "a04r", "b01r", "c02r"]

# for patient in patients:
#     print(f"Start training for patient {patient}")
#     X_stft = np.load(f"stft_individual/{patient}_stft_features.npy")
#     y = np.load(f"stft_individual/{patient}_labels.npy")

#     X_stft = process_data_for_conv2D(X_stft)
#     print(X_stft.shape)
#     print(y.shape)
#     path = f"weights/{patient}/"
#     if not os.path.exists(path):
#         print(f"Create directory {path}")
#         os.makedirs(path)
#     kfold_training(X_stft, y, k_fold, train_size, val_size, test_size, base_model_cnn, base_file_name, base_csv_name, path, learning_rate, epochs, batch_size, num_dense_units, num_classes, kernel_size, max_pool_kernel, conv_filters)

In [15]:
from tensorflow.keras.utils import plot_model 
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
